In [8]:
import os
import json
import pandas as pd
import traceback

In [9]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
load_dotenv()

True

In [10]:
key=os.getenv("OPENAI_API_KEY")

In [11]:
llm=ChatOpenAI(openai_api_key=key,model_name="gpt-3.5-turbo",temperature=0.5)

In [12]:
llm


ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x0000023ECD331100>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x0000023EAA0B6090>, root_client=<openai.OpenAI object at 0x0000023EA9F504A0>, root_async_client=<openai.AsyncOpenAI object at 0x0000023ECD331160>, temperature=0.5, model_kwargs={}, openai_api_key=SecretStr('**********'))

In [21]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [18]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [19]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be confirming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [22]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text","number","subject","tone","response_json"],
    template=TEMPLATE
)

In [27]:
quiz_chain=LLMChain(llm=llm,prompt=quiz_generation_prompt,output_key="quiz",verbose=True)

In [23]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [24]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)


In [25]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)


C:\Users\Jay\AppData\Local\Temp\ipykernel_14648\1121590683.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)


In [28]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [29]:
file_path=r"D:\mcqgenerator\data.txt"

In [30]:
file_path

'D:\\mcqgenerator\\data.txt'

In [31]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [103]:
print(TEXT)

What is LangChain?
LangChain is a framework designed to build applications powered by Large Language Models (LLMs), such as OpenAI's GPT-3.5 or GPT-4. It provides tools, abstractions, and integrations that make it easier to create applications like chatbots, question-answering systems, summarization tools, and more.

LangChain's core philosophy is modularity and chainability, meaning it helps developers break down complex workflows into smaller, manageable, and reusable components.

Key Components of LangChain
LLMs (Large Language Models):

LangChain supports multiple LLMs (e.g., OpenAI, Hugging Face, Cohere).
It provides wrappers to interact with these models and configure parameters like temperature, model_name, and max_tokens.
Prompts:

LangChain offers tools to manage, format, and template prompts effectively.
You can define reusable prompt templates and dynamically inject variables.
Example:

python
Copy
Edit
from langchain.prompts import PromptTemplate

template = "Translate the 

In [32]:
# Serialize the Python dictionary into a JSON-formatted string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [33]:
NUMBER=5 
SUBJECT="openAI"
TONE="Simple"

In [34]:
#https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking

#How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )

C:\Users\Jay\AppData\Local\Temp\ipykernel_14648\2654919708.py:5: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response=generate_evaluate_chain(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:OpenAI is an American artificial intelligence (AI) research organization founded in December 2015 and headquartered in San Francisco, California. Its stated mission is to develop "safe and beneficial" artificial general intelligence (AGI), which it defines as "highly autonomous systems that outperform humans at most economically valuable work".[5] As a leading organization in the ongoing AI boom,[6] OpenAI is known for the GPT family of large language models, the DALL-E series of text-to-image models, and a text-to-video model named Sora.[7][8] Its release of ChatGPT in November 2022 has been credited with catalyzing widespread interest in generative AI.

The organization consists of the non-profit OpenAI, Inc.,[9] registered in Delaware, and its for-profit subsidiary introduced in 2019, OpenAI Global, LLC.[10] Microsoft owns roughly 49% of OpenAI's equity, having invested US$13 

In [35]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:3453
Prompt Tokens:2782
Completion Tokens:671
Total Cost:0.0023975000000000003


In [36]:
quiz=response.get("quiz")
print(quiz)



{
"1": {
"mcq": "When was OpenAI founded?",
"options": {
"a": "2010",
"b": "2015",
"c": "2020",
"d": "2025"
},
"correct": "b"
},
"2": {
"mcq": "Who are the co-chairs of OpenAI?",
"options": {
"a": "Sam Altman and Elon Musk",
"b": "Greg Brockman and Trevor Blackwell",
"c": "Ilya Sutskever and Andrej Karpathy",
"d": "John Schulman and Wojciech Zaremba"
},
"correct": "a"
},
"3": {
"mcq": "Which AI model gained attention for its ability to generate human-like text?",
"options": {
"a": "DALL-E",
"b": "Sora",
"c": "GPT-2",
"d": "ChatGPT"
},
"correct": "c"
},
"4": {
"mcq": "Who invested US$13 billion in OpenAI and owns roughly 49% of its equity?",
"options": {
"a": "Elon Musk",
"b": "Sam Altman",
"c": "Microsoft",
"d": "Reid Hoffman"
},
"correct": "c"
},
"5": {
"mcq": "In what year did OpenAI release a public beta of 'OpenAI Gym'?",
"options": {
"a": "2015",
"b": "2016",
"c": "2017",
"d": "2018"
},
"correct": "b"
}
}


In [42]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [38]:
quiz_table_data

[]

In [39]:
quiz=pd.DataFrame(quiz_table_data)

In [40]:
quiz.to_csv("machinelearning.csv",index=False)

In [41]:
from datetime import datetime
datetime.now().strftime('%m_%d_%Y_%H_%M_%S')

'01_22_2025_21_08_45'

In [3]:
import os
print(os.environ.get("SSL_CERT_FILE"))

d:\mcqgenerator\env\Library\ssl\cacert.pem
